In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autorudata/data_datasetcar.csv
/kaggle/input/samplesubmis/sample_submission.csv
/kaggle/input/traindataset2/train150121.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string
from nltk.util import ngrams
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string
from nltk.util import ngrams
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [4]:
import re

In [5]:
sample_submission = pd.read_csv('../input/samplesubmis/sample_submission.csv')

In [6]:
sample_submission

,sell_id,price
0,1100575026,0
1,1100549428,0
2,1100658222,0
3,1100937408,0
4,1101037972,0
...,...,...
34681,1101369263,0
34682,1101369581,0
34683,1101364889,0
34684,1101362518,0


In [7]:
RANDOM_SEED = 42
VAL_SIZE   = 0.2

In [8]:
data = pd.read_csv('../input/autorudata/data_datasetcar.csv', low_memory=False) 

In [9]:
train = pd.read_csv('../input/traindataset2/train150121.csv', low_memory=False) 

In [10]:
y = train['price']

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224136 entries, 0 to 224135
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           224136 non-null  int64  
 1   brand                224136 non-null  int64  
 2   color                224136 non-null  int64  
 3   engineDisplacement   224136 non-null  float64
 4   enginePower          224136 non-null  float64
 5   fuelType             224136 non-null  int64  
 6   mileage              224136 non-null  int64  
 7   modelDate            224136 non-null  int64  
 8   model_name           224136 non-null  int64  
 9   numberOfDoors        224136 non-null  int64  
 10  productionDate       224136 non-null  int64  
 11  vehicleTransmission  224136 non-null  int64  
 12  vendor               224136 non-null  int64  
 13  Владельцы            224136 non-null  int64  
 14  ПТС                  224136 non-null  int64  
 15  Привод           

In [12]:
columns_drop = ['Unnamed: 0']
data = data.drop(columns_drop, axis=1)

In [13]:
for column in ['bodyType2', 'brand', 'color', 'fuelType', 'model_name', 'vendor', 'Владельцы', 'vehicleTransmission', 'Привод', 'age_intervals']:
    data[column] = data[column].astype('category').cat.codes

In [14]:
bin_cols = ['ПТС', 'Руль']
label_encoder = LabelEncoder()
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
data.head()

,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,...,Владельцы,ПТС,Привод,Руль,bodyType2,bT_length,Nalog,years_old,age_intervals,sample
0,8,14,1.2,105.0,0,74000,2013,710,5,2014,...,2,1,1,0,4,7,2625.0,7,1,0
1,8,17,1.6,110.0,0,60563,2017,710,5,2017,...,0,1,1,0,4,7,2750.0,4,1,0
2,8,13,1.8,152.0,0,88000,2013,954,5,2014,...,0,1,1,0,4,7,6840.0,7,1,0
3,8,7,1.6,110.0,0,95000,2013,710,5,2014,...,0,1,1,0,4,7,2750.0,7,1,0
4,8,1,1.8,152.0,0,58536,2008,710,5,2012,...,0,1,1,0,4,7,6840.0,9,2,0


In [15]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [16]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [18]:
np.log(y_train)

34318     12.785491
42375     12.206073
178827    15.287983
21807     13.975075
5154      13.304685
            ...    
119879    13.763164
103694    12.254863
131932    16.077065
146867    13.440090
121958    12.206073
Name: price, Length: 151560, dtype: float64

In [19]:
rfr = RandomForestRegressor(n_estimators=1000, max_depth = 15, max_features = 'log2', random_state = 42, oob_score=True,) 


In [ ]:
rfr.fit(X_train, np.log(y_train))

In [ ]:
predict_test = np.exp(rfr.predict(X_test)) 
predict_submission = np.exp(rfr.predict(X_sub))



In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_rfr1.csv', index=False)


# XGBRegressor


In [23]:
import xgboost as xgb

In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.005 ,max_depth=15, alpha=1,
                          n_estimators=5000)

In [ ]:
xg_reg.fit(X_train, np.log(y_train))

In [ ]:
predict_test = np.exp(xg_reg.predict(X_test)) 
predict_submission = np.exp(xg_reg.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_1606xg_final.csv', index=False)


In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.03 ,max_depth=14, alpha=1,
                          n_estimators=10000)

In [ ]:
xg_reg.fit(X_train, np.log(y_train))

In [ ]:
predict_test = np.exp(xg_reg.predict(X_test)) 
predict_submission = np.exp(xg_reg.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_1604xg.csv', index=False)


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_1502.csv', index=False)


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_xg_reg.csv', index=False)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [ ]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=5000)
gb.fit(X_train, np.log(y_train))

In [ ]:
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_1503gb.csv', index=False)


In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor


In [24]:
estimators = [
    ('xg_reg', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,learning_rate=0.007, max_depth=17, alpha=1,n_estimators=5000)),
    ('gb', GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=17, n_estimators=5000))
]

In [26]:
reg5 = StackingRegressor(estimators=estimators)


In [ ]:
reg5.fit(X_train, np.log(y_train))


In [ ]:
predict_test = np.exp(reg5.predict(X_test))
predict_submission = np.exp(reg5.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_st_xgb3.csv', index=False)
            

# StackingRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor


In [ ]:
estimators = [
    ('xg_reg', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,learning_rate=0.03, max_depth=12, alpha=1,n_estimators=5000)),
    ('gb', GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=12, n_estimators=5000)
)
]

In [ ]:
reg4 = StackingRegressor(estimators=estimators)
reg4.fit(X_train, np.log(y_train))


In [ ]:
predict_test = np.exp(reg4.predict(X_test))
predict_submission = np.exp(reg4.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_catxgb.csv', index=False)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
X_train.shape

In [ ]:
model = Sequential()
model.add(Dense(4737, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mape', metrics=['mse', 'mape'])

In [ ]:
model.fit(X_train, np.log(y_train), epochs=5, validation_data=(X_test, np.log(y_test)))

In [ ]:
preds = np.exp(model.predict(X_test))
preds_submission = np.exp(model.predict(X_sub))

In [ ]:
preds, y_test.values

In [ ]:
mape(y_test.values, preds)